In [3]:
from ImageSearch_VGG import modules
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imutils import paths
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2
import pickle
import random
import argparse

random.seed(42)
np.set_printoptions(threshold=np.inf)
img_size = (64,64)

#Initialize Data containers
data = []
labels = []
image_dir = os.path.join(os.getcwd(),"Images")

#Add image data to array over loop
for root,dirs,files in os.walk(image_dir):
    for file in files:
        if file.endswith(".pgm") or file.endswith(".jpg"):
            img_path = os.path.join(root, file)
            label = os.path.basename(root)
            labels.append(label)
            
            #Read image in Grayscale
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, img_size, interpolation=cv2.INTER_AREA)
            data.append(image)

#Data Scaling to range (0,1)
data = np.array(data, dtype = "float")/255.0
data = data[:,:,:,np.newaxis]             #Addition of extra Dimension for Grayscale images
labels = np.array(labels)

#Creating train test split
(train_x, test_x, train_y, test_y) = train_test_split(data, labels, test_size = 0.25, random_state=42)

#one hot encoding for lavels
lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y =lb.transform(test_y)

#Image data augmentation using keras
augment = ImageDataGenerator(rotation_range = 30, width_shift_range=0.1, height_shift_range= 0.1, 
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

#Initialize model using custom SmallVGGnet model
model = modules.smallVGGnet(width = 64, height = 64, depth =1, classes = len(lb.classes_)).build()
#model.summary()

#Initialize LR, epochs, batch size
INIT_LR =0.01
EPOCH = 150
BATCH = 32

#Initialize model and optimizer
opt = SGD(lr = INIT_LR, decay=INIT_LR/EPOCH)
model.compile(loss = 'categorical_crossentropy',optimizer=opt, metrics=["accuracy"])

#Train the network
#Tr = model.fit_generator(augment.flow(train_x, train_y, batch_size= BATCH, shuffle = True), validation_data=(test_x, test_y),steps_per_epoch = len(train_x)/BATCH, epochs = EPOCH)


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:

#Fit model to test data
predict = model.predict(test_x, batch_size=BATCH)
print(classification_report(test_y.argmax(axis =1), predict.argmax(axis =1), target_names = lb.classes_, labels = range(len(lb.classes_))))

#Plot training loss and accuracy
N = np.arange(0, EPOCH)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, Tr.history["acc"], label="accuracy")
plt.plot(N, Tr.history["val_acc"], label = "val_accuracy")
plt.plot(N, Tr.history["loss"], label= "Loss")
plt.plot(N, Tr.history["val_loss"], label="val_loss")
plt.legend()
plt.title("Model Metrics-Training loss and accuracy")
plt.xlabel("No. of epochs")
plt.ylabel("Training Loss and accuracy")


#Save model
model.save(os.path.join((os.getcwd()), "model\smallvggnet_model"), overwrite=True, include_optimizer=True)
f = open(os.path.join((os.getcwd()), "model\smallvggnet_LabelBinarizer"), 'wb')
f.write(pickle.dumps(lb))
f.close()

c:\users\nangoyal\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\nangoyal\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

          A1       0.00      0.00      0.00         3
         A10       0.00      0.00      0.00         5
        A100       0.00      0.00      0.00         4
        A101       0.00      0.00      0.00         4
        A102       0.00      0.00      0.00         4
        A103       0.00      0.00      0.00         2
        A104       0.00      0.00      0.00         3
        A105       0.00      0.00      0.00         2
        A106       0.00      0.00      0.00         6
        A107       0.00      0.00      0.00         1
        A108       0.00      0.00      0.00         1
        A109       0.00      0.00      0.00         4
         A11       0.00      0.00      0.00         2
        A110       0.00      0.00      0.00         4
        A111       0.00      0.00      0.00         1
        A112       0.00      0.00      0.00         3
        A113       0.00      0.00      0.00         2
        A114       0.00    

NameError: name 'Tr' is not defined

<Figure size 432x288 with 0 Axes>